In [3]:
# Requirement.
# 1. Connect to sql database using python.
import psycopg2
import pandas as pd
from IPython.display import display

hostname = 'localhost'
username = 'postgres'
password = 'password'
database = 'cards_ingest'
myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
myConnection.close()

In [50]:
# 2. Create dataframe by selecting all the cards_ingest.tran_fact
myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )

def do_query( conn, query : str, args : list ) :
    """
    execute postgresql query on connected database

    :param conn: psycopg2 connection
    :param query: postgresql query string 
    :param args: arguments for query string (see https://www.psycopg.org/docs/usage.html#the-problem-with-the-query-parameters)
    :return: pandas DataFrame for query result
    :rtype: pd.DataFrame
    """
    cur = conn.cursor()
    cur.execute( query, args )
    # pandas frame
    frame = None
    
    if cur.description : # checks for returned table (if it has a description, the query has returned a table)
        data = cur.fetchall()
        column_names = [i[0] for i in cur.description]
        frame = pd.DataFrame(data, columns=column_names)
    cur.close()
    
    return frame


query = "SELECT * FROM cards_ingest.tran_fact;"
tran_fact = do_query( myConnection, query, [] )
myConnection.close()

tran_fact

,tran_id,cust_id,stat_cd,tran_ammt,tran_date
0,102020,cust_101,NY,125.00,2022-01-01
1,102021,cust_101,NY,5125.00,2022-01-01
2,1020321,cust_101,NY,225.00,2022-02-01
3,1020121,cust_101,NY,4125.00,2022-02-03
4,1020222,cust_102,CA,6125.00,2022-01-01
5,1020223,cust_103,CA,7145.00,2022-01-01
6,1023023,cust_103,CA,7145.00,2022-04-01
7,1020123,cust_103,CA,7145.00,2022-03-01
8,1020223,cust_103,CA,7145.00,2022-03-02
9,102024,cust_104,TX,1023.00,2022-01-01


In [45]:
# 3. if the any state has NUll , replace it with TX
myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
query = "SELECT * FROM cards_ingest.tran_fact;"
tran_fact = do_query( myConnection, query, [] )

tran_fact.loc[(tran_fact['stat_cd']).isnull(), 'stat_cd'] = 'TX'
myConnection.close()

tran_fact

,tran_id,cust_id,stat_cd,tran_ammt,tran_date
0,102020,cust_101,NY,125.00,2022-01-01
1,102021,cust_101,NY,5125.00,2022-01-01
2,1020321,cust_101,NY,225.00,2022-02-01
3,1020121,cust_101,NY,4125.00,2022-02-03
4,1020222,cust_102,CA,6125.00,2022-01-01
5,1020223,cust_103,CA,7145.00,2022-01-01
6,1023023,cust_103,CA,7145.00,2022-04-01
7,1020123,cust_103,CA,7145.00,2022-03-01
8,1020223,cust_103,CA,7145.00,2022-03-02
9,102024,cust_104,TX,1023.00,2022-01-01


In [49]:
# 4. Create another col as commison = tran_ammt*.4
query = "SELECT * FROM cards_ingest.tran_fact;"
tran_fact = do_query( myConnection, query, [] 
tran_fact['commison'] = tran_fact['tran_ammt'].astype(float) * .4

tran_fact

TypeError: 'NoneType' object is not subscriptable

In [48]:
# 5. Create another table as cards_ingest.tran_fact with one extra col
myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
query = '''drop table cards_ingest.tran_fact_comm'''
tran_fact = do_query( myConnection, query, [])
query = '''create table cards_ingest.tran_fact_comm(
    tran_id int,
    cust_id varchar(10),
    stat_cd varchar(2),
    tran_ammt decimal(10,2),
    tran_date date,
    commison decimal(10,2)
    
);'''
do_query( myConnection, query, [])
myConnection.commit()
myConnection.close()

tran_fact




In [ ]:
# 6. Checkk if any null value still exists after the change.


In [ ]:
# 7. Load Data to the new table.


In [ ]:
# 8. Compare the record count from two tables to make sure that same number of records have been loaded.

In [8]:
# close connection
myConnection.close()

In [ ]:
'''Sample Code to connect:
#!/usr/bin/python

from __future__ import print_function

hostname = 'localhost'
username = 'username'
password = 'password'
database = 'dbname'

# Simple routine to run a query on a database and print the results:
def doQuery( conn ) :
    cur = conn.cursor()

    cur.execute( "SELECT fname, lname FROM employee" )

    for firstname, lastname in cur.fetchall() :
        print( firstname, lastname )


print( "Using psycopg2:" )
import psycopg2
myConnection = psycopg2.connect( host=hostname, user=username, password=password, dbname=database )
doQuery( myConnection )
myConnection.close()

print( "Using PyGreSQL (pgdb):" )
import pgdb
myConnection = pgdb.connect( host=hostname, user=username, password=password, database=database )
doQuery( myConnection )
myConnection.close()
'''